In [1]:
%tensorflow_version 2.x
import tensorflow as tf
from tensorflow import keras

TensorFlow 2.x selected.


In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [0]:
import zipfile
with zipfile.ZipFile('/content/drive/My Drive/pranav/intel-image-classification.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/data')

In [0]:
from tensorflow.keras.applications import vgg16

In [0]:
model = vgg16.VGG16(include_top=False, pooling='avg', input_shape = (150, 150, 3))

In [0]:
import numpy as np

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier


from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image as kimage

#First Cut

In [7]:
model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0     

In [0]:
for layer in model.layers:
  layer.trainable = False

In [9]:
train_processing = kimage.ImageDataGenerator(
    preprocessing_function=vgg16.preprocess_input)
train_generator = train_processing.flow_from_directory('/content/data/seg_train/seg_train', 
                                                       batch_size=102,
                                                       target_size=(150,150), shuffle=False)

test_processing = kimage.ImageDataGenerator(
    preprocessing_function=vgg16.preprocess_input)
test_generator = test_processing.flow_from_directory('/content/data/seg_test/seg_test', 
                                                     batch_size=100,
                                                     target_size=(150,150), shuffle=False)

Found 14034 images belonging to 6 classes.
Found 3000 images belonging to 6 classes.


In [10]:
%%time
for i in range(138): #round(14034/102)
  X, y = next(train_generator)
  if i == 0:
    X_train = model.predict(X)
    y_train = np.argmax(y, axis=1)
  else:
    X_train = np.concatenate((X_train, model.predict(X)))
    y_train = np.concatenate((y_train, np.argmax(y, axis=1)))
  if i%10==0 or i == 137:
    print(i, X_train.shape, y_train.shape)
np.save('/content/drive/My Drive/pranav/X_train_1', X_train)
np.save('/content/drive/My Drive/pranav/y_train_1', y_train)

0 (102, 512) (102,)
10 (1122, 512) (1122,)
20 (2142, 512) (2142,)
30 (3162, 512) (3162,)
40 (4182, 512) (4182,)
50 (5202, 512) (5202,)
60 (6222, 512) (6222,)
70 (7242, 512) (7242,)
80 (8262, 512) (8262,)
90 (9282, 512) (9282,)
100 (10302, 512) (10302,)
110 (11322, 512) (11322,)
120 (12342, 512) (12342,)
130 (13362, 512) (13362,)
137 (14034, 512) (14034,)
CPU times: user 1h 12min 18s, sys: 3min 21s, total: 1h 15min 40s
Wall time: 38min 42s


In [11]:
%%time
for i in range(30):
  X, y = next(test_generator)
  if i == 0:
    X_test = model.predict(X)
    y_test = np.argmax(y, axis=1)
  else:
    X_test = np.concatenate((X_test, model.predict(X)))
    y_test = np.concatenate((y_test, np.argmax(y, axis=1)))
  if i%10==0 or i == 29:
    print(i, X_test.shape, y_test.shape)
np.save('/content/drive/My Drive/pranav/X_test_1', X_test)
np.save('/content/drive/My Drive/pranav/y_test_1', y_test)

0 (100, 512) (100,)
10 (1100, 512) (1100,)
20 (2100, 512) (2100,)
29 (3000, 512) (3000,)
CPU times: user 14min 31s, sys: 42.2 s, total: 15min 13s
Wall time: 7min 47s


In [0]:
X_train = np.load('/content/drive/My Drive/pranav/X_train_1.npy')
y_train = np.load('/content/drive/My Drive/pranav/y_train_1.npy')
X_test  = np.load('/content/drive/My Drive/pranav/X_test_1.npy')
y_test  = np.load('/content/drive/My Drive/pranav/y_test_1.npy')

X_train, y_train = shuffle(X_train, y_train)
X_test, y_test = shuffle(X_test, y_test)

In [0]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2)

In [10]:
%%time
svclassifier = LinearSVC(dual=False, C=0.5, tol=1e-3).fit(X_train, y_train)
print(classification_report(y_train,svclassifier.predict(X_train)))
print(classification_report(y_val,svclassifier.predict(X_val)))

              precision    recall  f1-score   support

           0       0.96      0.96      0.96      1755
           1       1.00      1.00      1.00      1806
           2       0.90      0.89      0.89      1901
           3       0.90      0.91      0.90      2020
           4       0.98      0.98      0.98      1849
           5       0.96      0.96      0.96      1896

    accuracy                           0.95     11227
   macro avg       0.95      0.95      0.95     11227
weighted avg       0.95      0.95      0.95     11227

              precision    recall  f1-score   support

           0       0.90      0.91      0.90       436
           1       0.98      0.97      0.98       465
           2       0.84      0.82      0.83       503
           3       0.82      0.85      0.84       492
           4       0.92      0.92      0.92       425
           5       0.91      0.90      0.90       486

    accuracy                           0.89      2807
   macro avg       0.89

In [11]:
print(classification_report(y_test,svclassifier.predict(X_test)))

              precision    recall  f1-score   support

           0       0.89      0.88      0.89       437
           1       0.98      0.99      0.98       474
           2       0.84      0.80      0.82       553
           3       0.82      0.85      0.83       525
           4       0.93      0.95      0.94       510
           5       0.90      0.90      0.90       501

    accuracy                           0.89      3000
   macro avg       0.89      0.89      0.89      3000
weighted avg       0.89      0.89      0.89      3000



In [12]:
%%time
knnclassifier = KNeighborsClassifier(n_neighbors=3).fit(X_train, y_train)
print(classification_report(y_train,knnclassifier.predict(X_train)))
print(classification_report(y_val,knnclassifier.predict(X_val)))

              precision    recall  f1-score   support

           0       0.93      0.93      0.93      1755
           1       0.96      1.00      0.98      1806
           2       0.87      0.90      0.89      1901
           3       0.91      0.89      0.90      2020
           4       0.98      0.96      0.97      1849
           5       0.95      0.93      0.94      1896

    accuracy                           0.93     11227
   macro avg       0.93      0.93      0.93     11227
weighted avg       0.93      0.93      0.93     11227

              precision    recall  f1-score   support

           0       0.86      0.88      0.87       436
           1       0.94      0.98      0.96       465
           2       0.80      0.82      0.81       503
           3       0.81      0.77      0.79       492
           4       0.91      0.94      0.93       425
           5       0.89      0.85      0.87       486

    accuracy                           0.87      2807
   macro avg       0.87

In [13]:
print(classification_report(y_test,knnclassifier.predict(X_test)))

              precision    recall  f1-score   support

           0       0.86      0.87      0.86       437
           1       0.95      0.99      0.97       474
           2       0.80      0.82      0.81       553
           3       0.82      0.79      0.80       525
           4       0.94      0.94      0.94       510
           5       0.90      0.85      0.88       501

    accuracy                           0.88      3000
   macro avg       0.88      0.88      0.88      3000
weighted avg       0.88      0.88      0.88      3000



#Second Cut

In [14]:
x = GlobalAveragePooling2D()(model.layers[-6].output)
model_2 = Model(model.input, x)
model_2.compile(loss='mse',optimizer='sgd')
model_2.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0     

In [15]:
%%time

train_processing = kimage.ImageDataGenerator(
    preprocessing_function=vgg16.preprocess_input)
train_generator = train_processing.flow_from_directory('/content/data/seg_train/seg_train', 
                                                       batch_size=102,
                                                       target_size=(150,150), shuffle=False)

test_processing = kimage.ImageDataGenerator(
    preprocessing_function=vgg16.preprocess_input)
test_generator = test_processing.flow_from_directory('/content/data/seg_test/seg_test', 
                                                     batch_size=100,
                                                     target_size=(150,150), shuffle=False)

for i in range(138): #round(14034/102)
  X, y = next(train_generator)
  if i == 0:
    X_train = model_2.predict(X)
    y_train = np.argmax(y, axis=1)
  else:
    X_train = np.concatenate((X_train, model_2.predict(X)))
    y_train = np.concatenate((y_train, np.argmax(y, axis=1)))
  if i%10==0 or i == 137:
    print(i, X_train.shape, y_train.shape)
np.save('/content/drive/My Drive/pranav/X_train_2', X_train)
np.save('/content/drive/My Drive/pranav/y_train_2', y_train)

for i in range(30):
  X, y = next(test_generator)
  if i == 0:
    X_test = model_2.predict(X)
    y_test = np.argmax(y, axis=1)
  else:
    X_test = np.concatenate((X_test, model_2.predict(X)))
    y_test = np.concatenate((y_test, np.argmax(y, axis=1)))
  if i%10==0 or i == 29:
    print(i, X_test.shape, y_test.shape)
np.save('/content/drive/My Drive/pranav/X_test_2', X_test)
np.save('/content/drive/My Drive/pranav/y_test_2', y_test)

Found 14034 images belonging to 6 classes.
Found 3000 images belonging to 6 classes.
0 (102, 512) (102,)
10 (1122, 512) (1122,)
20 (2142, 512) (2142,)
30 (3162, 512) (3162,)
40 (4182, 512) (4182,)
50 (5202, 512) (5202,)
60 (6222, 512) (6222,)
70 (7242, 512) (7242,)
80 (8262, 512) (8262,)
90 (9282, 512) (9282,)
100 (10302, 512) (10302,)
110 (11322, 512) (11322,)
120 (12342, 512) (12342,)
130 (13362, 512) (13362,)
137 (14034, 512) (14034,)
0 (100, 512) (100,)
10 (1100, 512) (1100,)
20 (2100, 512) (2100,)
29 (3000, 512) (3000,)
CPU times: user 1h 13min 31s, sys: 3min 35s, total: 1h 17min 7s
Wall time: 39min 29s


In [0]:
X_train = np.load('/content/drive/My Drive/pranav/X_train_2.npy')
y_train = np.load('/content/drive/My Drive/pranav/y_train_2.npy')
X_test  = np.load('/content/drive/My Drive/pranav/X_test_2.npy')
y_test  = np.load('/content/drive/My Drive/pranav/y_test_2.npy')

X_train, y_train = shuffle(X_train, y_train)
X_test, y_test = shuffle(X_test, y_test)

In [0]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2)

In [30]:
%%time
svclassifier = LinearSVC(dual=False, C=1, tol=1e-3).fit(X_train, y_train)
print(classification_report(y_train,svclassifier.predict(X_train)))
print(classification_report(y_val,svclassifier.predict(X_val)))

              precision    recall  f1-score   support

           0       0.98      0.97      0.97      1759
           1       1.00      1.00      1.00      1795
           2       0.92      0.90      0.91      1947
           3       0.91      0.92      0.92      2017
           4       0.98      0.99      0.98      1802
           5       0.97      0.98      0.98      1907

    accuracy                           0.96     11227
   macro avg       0.96      0.96      0.96     11227
weighted avg       0.96      0.96      0.96     11227

              precision    recall  f1-score   support

           0       0.90      0.92      0.91       432
           1       0.98      0.98      0.98       476
           2       0.84      0.83      0.83       457
           3       0.85      0.87      0.86       495
           4       0.95      0.93      0.94       472
           5       0.92      0.90      0.91       475

    accuracy                           0.91      2807
   macro avg       0.91

In [31]:
print(classification_report(y_test,svclassifier.predict(X_test)))

              precision    recall  f1-score   support

           0       0.90      0.90      0.90       437
           1       0.99      1.00      0.99       474
           2       0.87      0.82      0.85       553
           3       0.83      0.88      0.86       525
           4       0.92      0.94      0.93       510
           5       0.92      0.91      0.91       501

    accuracy                           0.90      3000
   macro avg       0.91      0.91      0.91      3000
weighted avg       0.90      0.90      0.90      3000



In [21]:
%%time
knnclassifier = KNeighborsClassifier(n_neighbors=3).fit(X_train, y_train)
print(classification_report(y_train,knnclassifier.predict(X_train)))
print(classification_report(y_val,knnclassifier.predict(X_val)))

              precision    recall  f1-score   support

           0       0.97      0.89      0.93      1759
           1       0.98      0.99      0.99      1795
           2       0.86      0.94      0.90      1947
           3       0.92      0.89      0.91      2017
           4       0.99      0.92      0.95      1802
           5       0.90      0.97      0.94      1907

    accuracy                           0.93     11227
   macro avg       0.94      0.93      0.93     11227
weighted avg       0.94      0.93      0.93     11227

              precision    recall  f1-score   support

           0       0.94      0.77      0.85       432
           1       0.97      0.98      0.98       476
           2       0.76      0.87      0.81       457
           3       0.85      0.81      0.83       495
           4       0.95      0.87      0.91       472
           5       0.82      0.94      0.88       475

    accuracy                           0.88      2807
   macro avg       0.88

In [22]:
print(classification_report(y_test,knnclassifier.predict(X_test)))

              precision    recall  f1-score   support

           0       0.96      0.77      0.85       437
           1       0.98      1.00      0.99       474
           2       0.78      0.88      0.82       553
           3       0.85      0.78      0.81       525
           4       0.95      0.88      0.91       510
           5       0.82      0.96      0.88       501

    accuracy                           0.88      3000
   macro avg       0.89      0.88      0.88      3000
weighted avg       0.88      0.88      0.88      3000



#Third Cut

In [32]:
x = GlobalAveragePooling2D()(model.layers[-14].output)
model_3 = Model(model.input, x)
model_3.compile(loss='mse',optimizer='sgd')
model_3.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0   

In [33]:
%%time

train_processing = kimage.ImageDataGenerator(
    preprocessing_function=vgg16.preprocess_input)
train_generator = train_processing.flow_from_directory('/content/data/seg_train/seg_train', 
                                                       batch_size=102,
                                                       target_size=(150,150), shuffle=False)

test_processing = kimage.ImageDataGenerator(
    preprocessing_function=vgg16.preprocess_input)
test_generator = test_processing.flow_from_directory('/content/data/seg_test/seg_test', 
                                                     batch_size=100,
                                                     target_size=(150,150), shuffle=False)

for i in range(138): #round(14034/102)
  X, y = next(train_generator)
  if i == 0:
    X_train = model_3.predict(X)
    y_train = np.argmax(y, axis=1)
  else:
    X_train = np.concatenate((X_train, model_3.predict(X)))
    y_train = np.concatenate((y_train, np.argmax(y, axis=1)))
  if i%10==0 or i == 137:
    print(i, X_train.shape, y_train.shape)
np.save('/content/drive/My Drive/pranav/X_train_3', X_train)
np.save('/content/drive/My Drive/pranav/y_train_3', y_train)

for i in range(30):
  X, y = next(test_generator)
  if i == 0:
    X_test = model_3.predict(X)
    y_test = np.argmax(y, axis=1)
  else:
    X_test = np.concatenate((X_test, model_3.predict(X)))
    y_test = np.concatenate((y_test, np.argmax(y, axis=1)))
  if i%10==0 or i == 29:
    print(i, X_test.shape, y_test.shape)
np.save('/content/drive/My Drive/pranav/X_test_3', X_test)
np.save('/content/drive/My Drive/pranav/y_test_3', y_test)

Found 14034 images belonging to 6 classes.
Found 3000 images belonging to 6 classes.
0 (102, 128) (102,)
10 (1122, 128) (1122,)
20 (2142, 128) (2142,)
30 (3162, 128) (3162,)
40 (4182, 128) (4182,)
50 (5202, 128) (5202,)
60 (6222, 128) (6222,)
70 (7242, 128) (7242,)
80 (8262, 128) (8262,)
90 (9282, 128) (9282,)
100 (10302, 128) (10302,)
110 (11322, 128) (11322,)
120 (12342, 128) (12342,)
130 (13362, 128) (13362,)
137 (14034, 128) (14034,)
0 (100, 128) (100,)
10 (1100, 128) (1100,)
20 (2100, 128) (2100,)
29 (3000, 128) (3000,)
CPU times: user 33min 46s, sys: 1min 56s, total: 35min 42s
Wall time: 18min 24s


In [0]:
X_train = np.load('/content/drive/My Drive/pranav/X_train_3.npy')
y_train = np.load('/content/drive/My Drive/pranav/y_train_3.npy')
X_test  = np.load('/content/drive/My Drive/pranav/X_test_3.npy')
y_test  = np.load('/content/drive/My Drive/pranav/y_test_3.npy')

X_train, y_train = shuffle(X_train, y_train)
X_test, y_test = shuffle(X_test, y_test)

In [0]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2)

In [40]:
%%time
svclassifier = LinearSVC(dual=False, C=0.1, tol=1e-3).fit(X_train, y_train)
print(classification_report(y_train,svclassifier.predict(X_train)))
print(classification_report(y_val,svclassifier.predict(X_val)))

              precision    recall  f1-score   support

           0       0.84      0.80      0.82      1742
           1       0.96      0.97      0.97      1811
           2       0.82      0.79      0.80      1904
           3       0.78      0.81      0.80      2021
           4       0.85      0.85      0.85      1822
           5       0.82      0.85      0.83      1927

    accuracy                           0.85     11227
   macro avg       0.85      0.85      0.85     11227
weighted avg       0.85      0.85      0.85     11227

              precision    recall  f1-score   support

           0       0.83      0.79      0.81       449
           1       0.96      0.97      0.96       460
           2       0.79      0.80      0.80       500
           3       0.78      0.79      0.79       491
           4       0.83      0.82      0.82       452
           5       0.81      0.83      0.82       455

    accuracy                           0.83      2807
   macro avg       0.83

In [41]:
print(classification_report(y_test,svclassifier.predict(X_test)))

              precision    recall  f1-score   support

           0       0.83      0.78      0.81       437
           1       0.95      0.97      0.96       474
           2       0.81      0.77      0.79       553
           3       0.76      0.81      0.79       525
           4       0.84      0.84      0.84       510
           5       0.83      0.85      0.84       501

    accuracy                           0.83      3000
   macro avg       0.84      0.84      0.84      3000
weighted avg       0.84      0.83      0.83      3000

